# Style Transfer

Style Transfer直接利用了VGG-16，没有神经网络需要训练，直接拿一张噪音图开始做梯度下降。优点是任意两张图都可以做风格迁移，缺点就是，每拿两张图，都得train一遍

In [1]:
from neural_style_transfer import *

d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
 #
# fixed args - don't change these unless you have a good reason
#
default_resource_dir = os.path.join(os.path.dirname(__file__), "..", "Images")
content_images_dir = os.path.join(default_resource_dir, 'original')
style_images_dir = os.path.join(default_resource_dir, 'cartoon')

output_img_dir = os.path.join(default_resource_dir, 'styletransfer-output')

img_format = (4, '.jpg')  # saves images in the format: %04d.jpg

#
# modifiable args - feel free to play with these (only small subset is exposed by design to avoid cluttering)
# sorted so that the ones on the top are more likely to be changed than the ones on the bottom
#
parser = argparse.ArgumentParser()
parser.add_argument("--content_img_name", type=str, help="content image name", default='HeadBase.jpg')
parser.add_argument("--style_img_name", type=str, help="style image name", default='candy.jpg')
parser.add_argument("--height", type=int, help="height of content and style images", default=400)

parser.add_argument("--content_weight", type=float, help="weight factor for content loss", default=1e5)
parser.add_argument("--style_weight", type=float, help="weight factor for style loss", default=1e2)
parser.add_argument("--tv_weight", type=float, help="weight factor for total variation loss", default=1e2)

parser.add_argument("--optimizer", type=str, choices=['lbfgs', 'adam'], default='adam')
parser.add_argument("--model", type=str, choices=['vgg16', 'vgg19'], default='vgg19')
parser.add_argument("--init_method", type=str, choices=['random', 'content', 'style'], default='content')
parser.add_argument("--saving_freq", type=int, help="saving frequency for intermediate images (-1 means only final)", default=-1)
args = parser.parse_args()

# some values of weights that worked for figures.jpg, vg_starry_night.jpg (starting point for finding good images)
# once you understand what each one does it gets really easy -> also see README.md

# lbfgs, content init -> (cw, sw, tv) = (1e5, 3e4, 1e0)
# lbfgs, style   init -> (cw, sw, tv) = (1e5, 1e1, 1e-1)
# lbfgs, random  init -> (cw, sw, tv) = (1e5, 1e3, 1e0)

# adam, content init -> (cw, sw, tv, lr) = (1e5, 1e5, 1e-1, 1e1)
# adam, style   init -> (cw, sw, tv, lr) = (1e5, 1e2, 1e-1, 1e1)
# adam, random  init -> (cw, sw, tv, lr) = (1e5, 1e2, 1e-1, 1e1)
# 所有参数打包到config结构里
# just wrapping settings into a dictionary
optimization_config = dict()
for arg in vars(args):
    optimization_config[arg] = getattr(args, arg)
optimization_config['content_images_dir'] = content_images_dir
optimization_config['style_images_dir'] = style_images_dir
optimization_config['output_img_dir'] = output_img_dir
optimization_config['img_format'] = img_format

# original NST (Neural Style Transfer) algorithm (Gatys et al.)
results_path = neural_style_transfer(optimization_config)
